# Iceberg Classification Step 2: Model Training with Single GPU
The following code includes demonstration for:
- get data (``TFRecord``) from ``feature store``
- training with ``TFRecord`` on a single GPU

In [1]:
import tensorflow as tf
print("Version of TensorFlow is {}".format(tf.__version__))

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
37,application_1574692443370_0037,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.
Version of TensorFlow is 1.14.0

In [2]:
from hops import featurestore
from hops import experiment
from hops import tensorboard

In [3]:
def create_tf_dataset(tfrecord_path, name_list):
    dataset_dir = featurestore.get_training_dataset_path(tfrecord_path)
    input_files = tf.gfile.Glob(dataset_dir + "/part-r-*")
    dataset = tf.data.TFRecordDataset(input_files)
    # 'tf_record_schema' is needed because we need to parse a single example from all the TFRecords we have
    tf_record_schema = featurestore.get_training_dataset_tf_record_schema(tfrecord_path)

    def decode(example_proto):
        example = tf.parse_single_example(example_proto, tf_record_schema)
        x = tf.stack([example[name_list[0]], example[name_list[1]], example[name_list[2]]], axis=1)
        x = tf.reshape(x, [75, 75, 3])
        y = [tf.cast(example[name_list[3]], tf.float32)]
        return x,y
    
    dataset = dataset.map(decode).shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE).repeat(NUM_EPOCHS)
    return dataset

In [4]:
def create_model():
    model = tf.keras.models.Sequential()
    
    #Conv Layer 1
    model.add(tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=INPUT_SHAPE))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
    model.add(tf.keras.layers.Dropout(0.2))

    #Conv Layer 2
    model.add(tf.keras.layers.Conv2D(128, kernel_size=(3, 3), activation='relu' ))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(tf.keras.layers.Dropout(0.2))

    #Conv Layer 3
    model.add(tf.keras.layers.Conv2D(128, kernel_size=(3, 3), activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(tf.keras.layers.Dropout(0.2))

    #Conv Layer 4
    model.add(tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(tf.keras.layers.Dropout(0.2))

    #Flatten the data for upcoming dense layers
    model.add(tf.keras.layers.Flatten())

    #Dense Layers
    model.add(tf.keras.layers.Dense(512))
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.Dropout(0.2))

    #Dense Layer 2
    model.add(tf.keras.layers.Dense(256))
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.Dropout(0.2))

    #Sigmoid Layer
    model.add(tf.keras.layers.Dense(1))
    model.add(tf.keras.layers.Activation('sigmoid'))
    return model

In [5]:
def train_fn(learning_rate):
    train_tfrecord_path = "train_tfrecords_iceberg_classification_dataset"
    train_name_list = ["band_1", "band_2", "band_avg", "is_iceberg"]
    train_dataset = create_tf_dataset(train_tfrecord_path, train_name_list)
    
    test_tfrecord_path = "test_tfrecords_iceberg_classification_dataset"
    test_name_list = ["band_1", "band_2", "band_avg", "is_iceberg"]
    test_dataset = create_tf_dataset(test_tfrecord_path, test_name_list)
    
    
    model = create_model()
    model.compile(optimizer=tf.train.AdamOptimizer(learning_rate), loss='binary_crossentropy',  metrics=['accuracy'])
    tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tensorboard.logdir(), histogram_freq=0,
                             write_graph=True, write_images=True)
    callbacks = [tb_callback]
    callbacks.append(tf.keras.callbacks.ModelCheckpoint(tensorboard.logdir() + '/checkpoint-{epoch}.h5',
                    monitor='acc', verbose=0, save_best_only=True))
    history = model.fit(train_dataset, epochs=NUM_EPOCHS, steps_per_epoch=5, callbacks=callbacks)
    test_score = model.evaluate(test_dataset, callbacks=callbacks)
    
    return -float(history.history["loss"][-1])

In [1]:
# Hyperparameter for TFRecords

NUM_EPOCHS = 10 
BATCH_SIZE = 32
SHUFFLE_BUFFER_SIZE = 10000
# Hyperparameter for learning rate
LEARNING_RATE = 0.001
args_dict = {'learning_rate': [0.001, 0.0005, 0.0001, 0.005, 0.003, 0.009]}

# Input shape of the model
INPUT_SHAPE= (75, 75, 3)

Starting Spark application


KeyboardInterrupt: 

In [7]:
experiment.launch(train_fn, args_dict, name="Iceberg_classification_with_featurestore_and_TFRecords", local_logdir=True)

KeyboardInterrupt: 

## END of the Step2